In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import time

In [ ]:
img = cv2.imread("in_img.jpg")
%matplotlib inline
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb)

In [ ]:
def gaussian(x, sigma):
    calculation=(1 / (2 * np.pi * (sigma ** 2))) * np.exp(- ((x ** 2) / (2 * sigma ** 2)))
    return calculation

def difference_coordinates(x,y,z,t):
    distance=np.sqrt( ((z-x)**2) + ((t-y)**2))
    return distance

def difference_intensities(image,index,x,y,z,t):
    return(image[int(x)][int(y)][index]-image[z][t][index])

def I_filtered(img,temp,index,Wp,nx,ny,x,y,sigma_i,sigma_s):
    
    gaussian_intensity=gaussian(difference_intensities(img,index,nx,ny,x,y),sigma_i)
    gaussian_spatial=gaussian(difference_coordinates(nx,ny,x,y),sigma_s)
    w=gaussian_intensity*gaussian_spatial
    

    temp[index]+=(img[int(nx)][int(ny)][index])*w
    Wp[index]+=w
    return temp,Wp
    
def apply_filter(img,f_img,x,y,hl,filt_size,sigma_i,sigma_s):
    temp = [0,0,0]
    Wp=[0,0,0]
    i = 0
    while(i < filt_size):
        j = 0
        while(j < filt_size):
            neighbour_x = x -(hl-i)
            neighbour_y = y -(hl-j)
            if(neighbour_x >= len(img)):
                neighbour_x -= len(img)
            if(neighbour_y >= len(img[0])): 
                neighbour_y -= len(img[0])
            t=0
            while(t<len(temp)):
                temp,Wp=I_filtered(img,temp,t,Wp,neighbour_x,neighbour_y,x,y,sigma_i,sigma_s)
                t+=1
            j+=1
        i+=1
        
    k=0
    while(k<len(temp)):
        temp[k]=temp[k]/Wp[k]
        k+=1
        
    l=0
    while(l<len(temp)):
        f_img[x][y][l] = int(round(abs(temp[l])))
        l+=1
    
def visit(img,filt_size,sigma_i,sigma_s):
    half_len = filt_size//2
    filtered_img = np.zeros((img.shape))
    i = 0
    while(i < len(img)):
        j = 0
        while(j < len(img[0])):
            
            apply_filter(img,filtered_img,i,j,half_len,filt_size,sigma_i,sigma_s)
            j+=1
        i+=1
    return filtered_img



filtered_img_own=visit(img,7,15,18)
cv2.imwrite("filtered_image_own.png",filtered_img_own)

filtered_image_OpenCV = cv2.bilateralFilter(img, 7,15, 18)
cv2.imwrite("filtered_image_OpenCV.png", img)

-When I select (sigma_i=2 ,sigma_s=2) the filtered image don't change perceptibly.
-When I increase sigma_i value from 2 to 10 (sigma_i=10,sigma_s=2) we can see some difference on the filtered image.After increasing value of sigma intenstiy,the noise reduction is increased.
-Again I increase sigma_i value from 10 to 15(sigma_i=15,sigma_s=2).After perform that value the change is observable.Noises are reduce if we increase sigma_i value.
-After I change the sigma_s value (sigma_i=15,sigma_s=10).There is no visibly change on general but the edges are distinct.When I increase sigma_s value the edge-preserving is increase.

I select sigma_i=15 and sigma_s=18.Because when the sigma_i value is between this [15,20] two value denoising is almost same.I think sigma_s value is depend on the edge-preserving.So I select it for the same reason I think that,when increasing sigma_s the edge preserving also increase.
